# Twitter API v2: Finding Trending Hashtags
Author: Johanna Garthe <br>
Discover hashtags used by Twitter users and their volume on a desired date.

In [1]:
import tweepy
import config_twitter
import time
import pandas as pd
import re

client = tweepy.Client(bearer_token=config_twitter.BEARER_TOKEN, wait_on_rate_limit=True)

In [8]:
query = " "
starttime = '2022-03-01T00:00:00Z'
endtime = '2022-05-31T00:00:00Z'

In [9]:
tweets = []
for response in tweepy.Paginator(
   
   client.search_all_tweets,
   query = query + ' -is:retweet -is:reply lang:de',
   tweet_fields = ['created_at', 'public_metrics', 'text', 'id', 'entities'],
   start_time = starttime,
   end_time = endtime,
   max_results=500,     # Max number per page with system limit of 500
   limit=20):           # Request page account limit 300 requests per 15 minutes
   
   time.sleep(1)        # Full-archive 1 request / 1 second limit
   tweets.append(response)

result = []
user_dict = {}

for response in tweets:
    for tweet in response.data:
        tags = []
        try:
            [tags.append(', '.join(["#"+str(h['tag']) for h in tweet.entities['hashtags']]))]
        except:
            pass
        result.append({'text_id': tweet.id,
                    'text': tweet.text.replace('\n', ' ').replace('\r', ' '),
                    'text_hashtags': [tags if tags else None],
                    'created_at': tweet.created_at,
                    'retweets': tweet.public_metrics['retweet_count'],
                    'replies': tweet.public_metrics['reply_count'],
                    'likes': tweet.public_metrics['like_count'],
                    'quote_count': tweet.public_metrics['quote_count'],
                    })
df = pd.DataFrame(result)
print('Number of matching results:', len(df))
#df.to_csv('{}_{}.csv'.format(query, starttime, endtime, index=False, header=True))

#### Shows 25 most popular hashtags found in the text and their frequenceny

In [5]:
hashtags = df['text_hashtags']
tagslists = []
tagslist = []

for tags in hashtags:
    try:
        if tags is not None:
            for tag in tags:
                for t in tag:
                    tagslists.append(t.lower().split(','))
    except:
        pass

for list in tagslists:
    tagslist += list
df_hashtags = pd.DataFrame (tagslist, columns = ['Hashtags found in text'])
count = df_hashtags.pivot_table(columns=['Hashtags found in text'], aggfunc='size')

In [ ]:
count.sort_values(ascending=False).head(25)

# Twitter API v2: Full-Archive Tweet Counts
Check tweet volume using respective hashtag

In [ ]:
import tweepy
import config_twitter

def getClient():
    client = tweepy.Client(bearer_token=config_twitter.BEARER_TOKEN,
                           consumer_key=config_twitter.API_KEY,
                           consumer_secret=config_twitter.API_KEY_SECRET,
                           access_token=config_twitter.ACCESS_TOKEN,
                           access_token_secret=config_twitter.ACCESS_TOKEN_SECRET)
    return client

def countTweets(query,starttime,endtime):
    client = getClient()
    counts = client.get_all_tweets_count(
        query=query,
        start_time = starttime,
        end_time = endtime,
        granularity='day',
        #next_token=nexttoken
        )
    for count in counts:
        print(count)

In [ ]:
# Define query
searchterms = " "
query = searchterms+' -is:retweet -is:reply lang:de'
#nexttoken = " "
countTweets(query, starttime='2022-02-24T00:00:00Z', endtime='2022-08-31T00:00:00Z ')